# Week 8 Homework

***Due (pushed to your GitHub branch) on 11/8 by 11:59 pm***

## Mark duplicates

Use Picard Tools to mark the duplicates of the HISAT and STAR alignments. Store the logs in a new directory.

In [4]:
# !mkdir placenta_logs
# !mkdir placenta_dedup

!picard MarkDuplicates -I ../6_alignment/placenta_alignment/star -M placenta_logs/star_duplicates -O placenta_dedup/star.bam --VALIDATION_STRINGENCY SILENT
!picard MarkDuplicates -I ../6_alignment/placenta_alignment/hisat/merged.bam -M placenta_logs/hisat_duplicates -O placenta_dedup/hisat.bam --VALIDATION_STRINGENCY SILENT


mkdir: cannot create directory ‘placenta_logs’: File exists


## Counting

Use featureCounts to count reads. You may need to explicitly set the features and meta-features with `-t` and `-g`.

In [ ]:
!featureCounts -T 32 \
    placenta_dedup/star.bam \
    -T 32 \
    --byReadGroup \
    -s 1 \
    --ignoreDup \
    -M \
    --fraction \
    -a ../2_genome_exploration/placenta_genome/placenta_annotations.gtf \
    -o placenta_star_counts.tsv \
    --verbose


!featureCounts -T 32 \
    placenta_dedup/hisat.bam \
    -T 32 \
    --byReadGroup \
    -s 1 \
    --ignoreDup \
    -M \
    --fraction \
    -a ../2_genome_exploration/placenta_genome/placenta_annotations.gtf \
    -o placenta_hisat_counts.tsv \
    --verbose 

## Report

Use MultiQC to aggregate a report that incorporates all steps of the pipeline.

In [ ]:
!multiqc --force -d ../5_fastq/fastq_placenta/placenta_qc/ ../5_fastq/fastq_placenta/placenta_qc_trimmed/ ../6_alignment ../7_alignment_qc .

## Snakemake Pipeline (optional)

1. Copy your `Snakefile` and `config.yaml` from `7_alignment_qc` to `8_counting`.  
2. In the `Snakefile`, add the following rules:  
    a. `mark_duplicates` - use Picard Tools to mark the duplicate reads in the STAR and HISAT alignments.  
    b. `count` - use featureCounts to count reads in the alignments with duplicates marked.  
    c. `report` - use `multiqc` to aggregate all QC data into a single report. This should be an update of the report from the previous week's homework.  

This `Snakefile` should not use any `wrappers` because we already have the relevant software installed in the `biol343` conda environment. The pipeline should run to completion when any instructor or classmate runs `snakemake --use-conda`.